<a href="https://colab.research.google.com/github/eliballer/eliballer/blob/main/Detector_Program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow.lite as tflite

# Load the model
interpreter = tflite.Interpreter(model_path="/content/best_float32.tflite")
interpreter.allocate_tensors()


In [3]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input Details:", input_details)
print("Output Details:", output_details)


Input Details: [{'name': 'images', 'index': 0, 'shape': array([  1, 640, 640,   3], dtype=int32), 'shape_signature': array([  1, 640, 640,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output Details: [{'name': 'Identity', 'index': 405, 'shape': array([   1,    9, 8400], dtype=int32), 'shape_signature': array([   1,    9, 8400], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [9]:
from PIL import Image
import numpy as np

# Load and preprocess the image
input_shape = input_details[0]['shape']  # e.g., [1, 224, 224, 3]
image = Image.open("/content/Img.jpg").resize((input_shape[1], input_shape[2]))
input_data = np.expand_dims(np.array(image, dtype=np.float32) / 255.0, axis=0)
input_data = input_data.astype(input_details[0]['dtype'])


In [10]:
# Set input tensor
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run inference
interpreter.invoke()

# Get output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])
print("Model Output:", output_data)


Model Output: [[[7.2077401e-03 1.8808380e-02 3.1744108e-02 ... 8.8094109e-01
   9.3028915e-01 9.7601056e-01]
  [4.1203089e-03 1.5002787e-03 8.3169006e-03 ... 9.6305019e-01
   9.6296072e-01 9.7247255e-01]
  [1.9348612e-01 1.8510208e-01 1.8138893e-01 ... 7.5345242e-01
   7.4589610e-01 7.4643999e-01]
  ...
  [1.3092502e-04 1.3413782e-04 1.3628369e-04 ... 1.9228287e-03
   1.9220319e-03 1.9548861e-03]
  [1.3853041e-04 1.4133357e-04 1.4506378e-04 ... 2.4314218e-03
   2.4123667e-03 2.4783222e-03]
  [1.5984119e-04 1.5285258e-04 1.5222220e-04 ... 1.6503589e-03
   1.6451065e-03 1.7301192e-03]]]


In [11]:
import numpy as np

# Example mapping of class IDs to XAF denominations
denomination_mapping = {
    0: 500,    # Class ID 0 represents 500 XAF
    1: 1000,   # Class ID 1 represents 1000 XAF
    2: 2000,   # Class ID 2 represents 2000 XAF
    3: 5000,   # Class ID 3 represents 5000 XAF
    4: 10000   # Class ID 4 represents 10000 XAF
}

# Post-process model output
output_shape = output_data.shape
print("Output Shape:", output_shape)

# Flatten the output if it's multidimensional
flat_output = np.reshape(output_data, (-1, output_shape[-1]))  # Flatten all except the last dimension

# Find the predicted class and confidence for each region
total_amount = 0
for row in flat_output:
    # Apply softmax to convert logits to probabilities
    probabilities = np.exp(row) / np.sum(np.exp(row))

    # Find the predicted class and its confidence
    predicted_class = np.argmax(probabilities)
    confidence = probabilities[predicted_class]

    # Add to total amount if confidence is above threshold
    if confidence > 0.5:  # Confidence threshold
        total_amount += denomination_mapping.get(predicted_class, 0)

print(f"Total Amount Detected: {total_amount} XAF")

Output Shape: (1, 9, 8400)
Total Amount Detected: 0 XAF


In [ ]:
import numpy as np

# Example mapping of class IDs to XAF denominations
denomination_mapping = {
    0: 500,    # Class ID 0 represents 500 XAF
    1: 1000,   # Class ID 1 represents 1000 XAF
    2: 2000,   # Class ID 2 represents 2000 XAF
    3: 5000,   # Class ID 3 represents 5000 XAF
    4: 10000   # Class ID 4 represents 10000 XAF
}

# Post-process model output
output_shape = output_data.shape
print("Output Shape:", output_shape)

# Flatten the output if it's multidimensional
flat_output = np.reshape(output_data, (-1, output_shape[-1]))  # Flatten all except the last dimension

# Find the predicted class and confidence for each region
total_amount = 0
for row in flat_output:
    # Apply softmax to convert logits to probabilities
    probabilities = np.exp(row) / np.sum(np.exp(row))

    # Find the predicted class and its confidence
    predicted_class = np.argmax(probabilities)
    confidence = probabilities[predicted_class]

    # Add to total amount if confidence is above threshold
    if confidence > 0.5:  # Confidence threshold
        total_amount += denomination_mapping.get(predicted_class, 0)

print(f"Total Amount Detected: {total_amount} XAF")